""""

1. Tried pulling directly from the website source api, got my ass kicked
2. Skywalk API is working and they fixed some requests
3. Testing dashboard combined with Apartments.com data-puller

""""

In [132]:
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import Input, Output, dcc, html
import dash_bootstrap_components as dbc
import pandas as pd

import censusdata
import us
import pandas as pd

In [16]:
import requests
from bs4 import BeautifulSoup
import json

appfolio_api_key = "wc9DeJ9UHoJKkEsgr9CN4W.8AnBp5DgrPE2HNdNMQ1HxwDlBkJnYRZDyGvletEznkE"
payload = {"X-API-KEY" : appfolio_api_key}
params = { "search": "?propertyId=51",}
cf_params = { "search": "?propertyId=51&accounting=cash&dateRange=this_month_to_date",}




In [19]:
unit_vacancies_request = requests.get('https://api.skywalkapi.com/v1/unit-vacancies', headers=payload, params = params)
print(unit_vacancies_request.raise_for_status())
unit_vacancies = json.loads(unit_vacancies_request.content)
unit_vacancies

None


{'meta': {'status': 'updating',
  'path': '/v1/unit-vacancies',
  'lastUpdated': 1642526923,
  'jobId': 'f93f8c2b-9bfd-4502-b419-a27d8f6629e1',
  'count': 8,
  'query': {}},
 'links': {},
 'data': [{'appfolioId': '2266',
   'name': '302 R',
   'description': '',
   'availableOn': '01/07/2022',
   'advertisedRent': '1,300.00',
   'computedMarketRent': '1,300.00',
   'amenities': None,
   'amenitiesPrice': '0.00',
   'bedAndBath': '3/1.00',
   'sqFt': '1,100',
   'daysVacant': 21,
   'lastMoveIn': '12/29/2020',
   'lastMoveOut': '12/28/2021',
   'lastRent': None,
   'newRent': None,
   'nextMoveIn': None,
   'postedToInternet': True,
   'postedToWebsite': True,
   'readyForShowingOn': '01/07/2022',
   'rentReady': False,
   'schdRent': '1,300.00',
   'status': 'Vacant-Unrented',
   'turnTargetDate': '01/07/2022',
   'type': 'Sunflower (3B/1B)',
   'tags': None,
   'property': {'appfolioId': '51',
    'label': 'The Sterling at San Marco - 2943 Spring Park Rd Jacksonville, FL 32207'},
   '

In [20]:
cash_flow_request = requests.get('https://api.skywalkapi.com/v1/cash-flow', headers=payload, params = cf_params)
print(cash_flow_request.raise_for_status())
cashflow = json.loads(cash_flow_request.content)
cashflow

None


{'meta': {'status': 'updating',
  'path': '/v1/cash-flow',
  'lastUpdated': None,
  'jobId': '7810cfc1-c29a-4917-9626-84af5fbadde2',
  'query': {'accounting': 'cash',
   'ownerPercentBased': 'No',
   'dateRange': 'this_year_to_date',
   'dateStart': '01/01/2022',
   'dateEnd': '02/23/2022'}}}

# Dashboard

In [126]:
#zillow rent data
rent_df = pd.read_csv("https://files.zillowstatic.com/research/public_csvs/zori/Metro_ZORI_AllHomesPlusMultifamily_SSA.csv?t=1644973146")
rent_df.drop(columns= ["RegionID", "SizeRank"], inplace=True)
rent_df.head()

,RegionName,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,...,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01
0,United States,1294.0,1299,1305.0,1310,1315,1321,1326,1331,1337,...,1708,1729,1750,1771,1793,1815.0,1837.0,1859,1881,1904
1,"New York, NY",2256.0,2266,2276.0,2286,2296,2306,2316,2326,2336,...,2526,2556,2586,2617,2648,2680.0,2712.0,2746,2780,2814
2,"Los Angeles-Long Beach-Anaheim, CA",1792.0,1803,1814.0,1825,1835,1846,1857,1868,1878,...,2497,2524,2550,2577,2604,2631.0,2659.0,2687,2716,2744
3,"Chicago, IL",1411.0,1415,1419.0,1423,1426,1430,1433,1437,1440,...,1649,1662,1675,1688,1702,1716.0,1729.0,1744,1758,1772
4,"Dallas-Fort Worth, TX",1128.0,1132,1135.0,1139,1143,1146,1150,1154,1157,...,1534,1556,1578,1600,1622,1645.0,1667.0,1690,1714,1737


In [127]:
reframed_rent_df = pd.DataFrame(rent_df.set_index("RegionName").stack()).reset_index()
reframed_rent_df.columns = ["RegionName", "Date", "ZRent"]
reframed_rent_df.RegionName = reframed_rent_df.RegionName.str.replace(",", "")

# housing from ZRent
housing_df = pd.read_csv("https://files.zillowstatic.com/research/public_csvs/zhvi/Metro_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv?t=1645574925")
housing_df.drop(columns= ["RegionID", "SizeRank", "RegionType", "StateName"], inplace=True)

reframed_housing_df = pd.DataFrame(housing_df.set_index("RegionName").stack()).reset_index()
reframed_housing_df.columns = ["RegionName", "Date", "ZEstimate"]
reframed_housing_df.RegionName = reframed_housing_df.RegionName.str.replace(",", "")
reframed_housing_df.Date = reframed_housing_df.Date.str[:-3]

reframe_df = pd.merge(reframed_rent_df, reframed_housing_df, how = "left", left_on = ["RegionName", "Date"], right_on = ["RegionName", "Date"])

reindexed_reframed_df = reframe_df.set_index(["RegionName", "Date"])

# adding percentage change
reindexed_reframed_df["rent_pct_change"] = reindexed_reframed_df["ZRent"].pct_change()
reindexed_reframed_df["home_value_pct_change"] = reindexed_reframed_df["ZEstimate"].pct_change()


reindexed_reframed_df.reset_index(inplace = True)
reindexed_reframed_df = reindexed_reframed_df[reindexed_reframed_df.Date != "2014-01"]

reindexed_reframed_df

,RegionName,Date,ZRent
0,United States,2014-01,1294.0
1,United States,2014-02,1299.0
2,United States,2014-03,1305.0
3,United States,2014-04,1310.0
4,United States,2014-05,1315.0
...,...,...,...
9880,College Station TX,2021-09,1524.0
9881,College Station TX,2021-10,1532.0
9882,College Station TX,2021-11,1541.0
9883,College Station TX,2021-12,1550.0


In [155]:
true_housing_df = pd.read_csv("https://files.zillowstatic.com/research/public_csvs/mlp/Metro_mlp_uc_sfrcondo_sm_month.csv?t=1646178611")
true_housing_df.drop(columns= ["RegionID", "SizeRank", "RegionType", "StateName"], inplace=True)
reframed_true_housing_df = pd.DataFrame(true_housing_df.set_index("RegionName").stack()).reset_index()

reframed_true_housing_df.columns = ["RegionName", "Date", "ZEstimate"]
reframed_true_housing_df.RegionName = reframed_true_housing_df.RegionName.str.replace(",", "")
reframed_true_housing_df.Date = reframed_true_housing_df.Date.str[:-3]

reframed_true_housing_df


,RegionName,level_1,0
0,United States,2018-01-31,271617.0
1,United States,2018-02-28,272983.0
2,United States,2018-03-31,279333.0
3,United States,2018-04-30,288667.0
4,United States,2018-05-31,295633.0
...,...,...,...
4503,"Fort Collins, CO",2020-08-31,477317.0
4504,"Fort Collins, CO",2020-09-30,473000.0
4505,"Fort Collins, CO",2020-10-31,466667.0
4506,"Fort Collins, CO",2020-11-30,463000.0


In [ ]:
def graph_layout(id_name, title):
    return html.Div(children=[
        
                html.Br(),
                html.Label(title),
                dcc.Graph(
                    id=id_name,
                )],
            style=style, draggable=True)

def create_sub_plot(filtered_df, dependent_variable):
    fig = px.line(filtered_df, x="Date", y=dependent_variable, color="RegionName", hover_name="RegionName")
    fig.update_layout(transition_duration=500, template = template)
    return fig

app = JupyterDash(__name__, 
            external_stylesheets=[dbc.themes.BOOTSTRAP],
            meta_tags=[
        {"name": "viewport", "content": "width=device-width, initial-scale=1"}
    ],)
del app.config._read_only["requests_pathname_prefix"]
server = app.server


template = "simple_white"
style = {'width': '49%', 'display': 'inline-block',"text-align" : "center",}
 

app.layout = html.Div([
    html.Div(children=[
        
        html.Br(),
        html.Label('Pick Cities to compare'),
        dcc.Dropdown(options = reindexed_reframed_df.RegionName.unique(),
                     value = ["United States", "Jacksonville FL"],
                     multi=True, 
                     id="cities_to_graph"),
       
    ]),
    
    graph_layout('Zrent', 'Graph of ZRents'), 
    graph_layout('Zrent_pct_change', 'Graph of ZRents Change'), 
    graph_layout('ZEst', 'Graph of ZEstimates'), 
    graph_layout('ZEst_pct_change', 'Graph of ZEstimates Change')

    ])

@app.callback(
    Output('Zrent', 'figure'),
    Output('Zrent_pct_change', 'figure'),
    Output('ZEst', 'figure'),
    Output('ZEst_pct_change', 'figure'),

    Input('cities_to_graph', 'value'))

def update_figure(selected_cities):
    
    print("selected_cities", selected_cities)
    if not selected_cities:
        selected_cities = ["United States"]
        
    filtered_df = reindexed_reframed_df[reindexed_reframed_df.RegionName.isin(selected_cities)]  
    
    fig1 = create_sub_plot(filtered_df, "ZRent")
    fig2 = create_sub_plot(filtered_df, "rent_pct_change")
    fig3 = create_sub_plot(filtered_df, "ZEstimate")
    fig4 = create_sub_plot(filtered_df, "home_value_pct_change")
    
    return fig1, fig2, fig3, fig4
    
if __name__ == '__main__':
    app.run_server(debug=False)

### Census Data

In [6]:
censusdata.search('acs5', 2019, 'label', 'total population')

population_in_occupied_units = "B25026_001E"
median_value = "B25077_001E"
gross_rent_total = "B25031_001E"

census_columns_to_download = [population_in_occupied_units, median_value, gross_rent_total]
years_to_download = [2015, 2016, 2017, 2018, 2019]

aggregate_census_df = pd.DataFrame()


for year in years_to_download:
    for state in us.states.STATES:
        new_data_df = censusdata.download('acs5', year, censusdata.censusgeo([('state', state.fips)]), census_columns_to_download)
        #new_data_df = censusdata.download('acs5', year, censusdata.censusgeo([('state', state.fips), ("tract", "*")]), census_columns)
        new_data_df["year"] = year
        aggregate_census_df = aggregate_census_df.append(new_data_df, ignore_index=False)

aggregate_census_df

,B25026_001E,B25077_001E,B25031_001E,year
"Boston-Worcester-Providence, MA-RI-NH-CT CSA: Summary level: 330, combined statistical area:148",7778988,315000,1089.0,2015
"Cedar Rapids-Iowa City, IA CSA: Summary level: 330, combined statistical area:168",409189,156700,752.0,2015
"Buffalo-Cheektowaga, NY CSA: Summary level: 330, combined statistical area:160",1179833,122300,720.0,2015
"Cape Coral-Fort Myers-Naples, FL CSA: Summary level: 330, combined statistical area:162",990613,187000,981.0,2015
"Cape Girardeau-Sikeston, MO-IL CSA: Summary level: 330, combined statistical area:164",131795,118100,636.0,2015
...,...,...,...,...
"Bend-Prineville, OR CSA: Summary level: 330, combined statistical area:140",207815,350300,1169.0,2019
"Birmingham-Hoover-Talladega, AL CSA: Summary level: 330, combined statistical area:142",1283354,156500,859.0,2019
"Bloomington-Bedford, IN CSA: Summary level: 330, combined statistical area:144",196728,153800,872.0,2019
"Bloomington-Pontiac, IL CSA: Summary level: 330, combined statistical area:145",198361,157800,826.0,2019


In [32]:
new_aggregate_census_df = new_aggregate_census_df.reset_index()
new_aggregate_census_df.columns = ["RegionName", "population_in_occupied_units", "median_value", "gross_rent_total", "year"]
new_aggregate_census_df

,RegionName,population_in_occupied_units,median_value,gross_rent_total,year
0,"Beaumont-Port Arthur, TX Metro Area: Summary l...",388180,97700,759.0,2015
1,"Beaver Dam, WI Micro Area: Summary level: 310,...",83689,153100,761.0,2015
2,"Beckley, WV Metro Area: Summary level: 310, me...",118620,90100,625.0,2015
3,"Bedford, IN Micro Area: Summary level: 310, me...",45102,100500,637.0,2015
4,"Beeville, TX Micro Area: Summary level: 310, m...",24934,72900,802.0,2015
...,...,...,...,...,...
4697,"Opelousas, LA Micro Area: Summary level: 310, ...",81917,108400,652.0,2019
4698,"Orangeburg, SC Micro Area: Summary level: 310,...",84855,92700,687.0,2019
4699,"Orlando-Kissimmee-Sanford, FL Metro Area: Summ...",2467571,221600,1210.0,2019
4700,"Fort Dodge, IA Micro Area: Summary level: 310,...",33672,99100,642.0,2019


In [105]:
new_aggregate_census_df[new_aggregate_census_df.RegionName.str.contains("Austin")]

,RegionName,population_in_occupied_units,median_value,gross_rent_total,year
404,Austin MN,38641,111700,697.0,2015
405,Austin-Round Rock TX,1848712,208700,1044.0,2015
1790,Austin MN,38616,114100,693.0,2016
1791,Austin-Round Rock TX,1901359,223500,1098.0,2016
2209,Austin MN,38719,114700,704.0,2017
2210,Austin-Round Rock TX,1958839,240500,1155.0,2017
3062,Austin MN,38870,120500,709.0,2018
3063,Austin-Round Rock TX,2016787,261400,1217.0,2018
4161,Austin MN,39017,123900,716.0,2019
4162,Austin-Round Rock-Georgetown TX,2071653,281500,1273.0,2019


127.0.0.1 - - [23/Feb/2022 22:31:23] "POST /_dash-update-component HTTP/1.1" 200 -


selected_cities ['Orlando-Kissimmee-Sanford FL', 'New York-Newark-Jersey City NY-NJ-PA', 'Tampa-St. Petersburg-Clearwater FL', 'Bozeman MT', 'Austin-Round Rock TX']


In [91]:
new_aggregate_census_df.RegionName = new_aggregate_census_df.RegionName.astype(str).apply(lambda x: x[:-98])
new_aggregate_census_df.RegionName = new_aggregate_census_df.RegionName.str.replace(',', "")
# recoding names
new_aggregate_census_df.RegionName = new_aggregate_census_df.RegionName.str.replace("Austin-Round Rock TX", "Austin-Round Rock-Georgetown TX")

In [117]:
new_aggregate_census_df.sort_values(["RegionName", "year"]).population_in_occupied_units.pct_change()

349          NaN
1733    0.012096
2102    0.006331
2972    0.006901
4104    0.002761
          ...   
928    -0.830324
1688    0.001751
1991    0.005525
3194   -0.003060
4696   -0.004674
Name: population_in_occupied_units, Length: 4702, dtype: float64

127.0.0.1 - - [24/Feb/2022 11:35:24] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2022 11:35:24] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2022 11:35:25] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [24/Feb/2022 11:35:25] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [24/Feb/2022 11:35:25] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [24/Feb/2022 11:35:25] "POST /_dash-update-component HTTP/1.1" 200 -


selected_cities ['Orlando-Kissimmee-Sanford FL']


In [ ]:
### PLOTTING ###
app = JupyterDash(__name__)
del app.config._read_only["requests_pathname_prefix"]
server = app.server

template = "simple_white"
style = {'width': '49%', 'display': 'inline-block',"text-align" : "center",}



app.layout = html.Div([
    html.Div(children=[
        html.Br(),
        html.Label('Pick Cities to compare'),
        dcc.Dropdown(options = new_aggregate_census_df.RegionName.unique(),
                     value = ["Orlando-Kissimmee-Sanford FL"],
                     multi=True, 
                     id="cities_to_graph"),
       
    ]),
    
    graph_layout('population_in_occupied_units', 'Graph of Population'), 
    #graph_layout('population_in_occupied_units', 'Graph of Population'), 

    graph_layout('median_value', 'Graph of Median Values'), 
    #graph_layout('median_value', 'Graph of Median Values'), 

    graph_layout('gross_rent_total', 'Graph of Gross Rents')

    ])

@app.callback(
    Output('population_in_occupied_units', 'figure'),
    Output('median_value', 'figure'),
    Output('gross_rent_total', 'figure'),

    Input('cities_to_graph', 'value'))

def update_figure(selected_cities):
    print("selected_cities", selected_cities)
    if not selected_cities:
        selected_cities = ["United States"]
        
    filtered_df = new_aggregate_census_df[new_aggregate_census_df.RegionName.isin(selected_cities)]   
    
    fig1 = create_sub_plot(filtered_df, "population_in_occupied_units")
    fig2 = create_sub_plot(filtered_df, "median_value")
    fig3 =create_sub_plot(filtered_df, "gross_rent_total")
    
    return fig1, fig2, fig3
    
if __name__ == '__main__':
    app.run_server(debug=False)

127.0.0.1 - - [01/Mar/2022 17:38:52] "GET /_shutdown_1816f2bf-c1de-4c69-b743-1e8171fb5496 HTTP/1.1" 200 -
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Mar/2022 17:38:52] "GET /_alive_1816f2bf-c1de-4c69-b743-1e8171fb5496 HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8050/


127.0.0.1 - - [01/Mar/2022 17:38:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2022 17:38:54] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2022 17:38:54] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2022 17:38:54] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Mar/2022 17:38:54] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Mar/2022 17:38:54] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [01/Mar/2022 17:38:54] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Mar/2022 17:38:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2022 17:38:54] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2022 17:38:54] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2022 17:38:54] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Mar/2022 17:38:54] "GET /_dash-component-suite

### Add apartment list too

In [122]:
apt_list_df = pd.read_csv("https://assets.ctfassets.net/jeox55pd4d8n/2QcueWduZDvVORgKJtXveW/407262141f9b1669748e6e147f112c13/Apartment_List_Rent_Estimates_City_2022_2.csv")
apt_list_df.dropna(inplace=True)

In [124]:
apt_list_df.drop("FIPS_Code", "Population")

,City_Name,FIPS_Code,Population,Bedroom_Size,2017_01,2017_02,2017_03,2017_04,2017_05,2017_06,...,2021_05,2021_06,2021_07,2021_08,2021_09,2021_10,2021_11,2021_12,2022_01,2022_02
0,"Birmingham, AL",107000.0,212021.0,_Overall,825.00,829.00,835.00,843.00,852.00,854.00,...,961.00,980.00,1007.00,1019.00,1026.00,1019.00,1014.00,1019.00,1031.00,1041.00
1,"Birmingham, AL",107000.0,212021.0,_Studio,730.00,734.00,739.00,746.00,753.00,756.00,...,850.00,867.00,890.00,902.00,907.00,902.00,897.00,901.00,912.00,921.00
2,"Birmingham, AL",107000.0,212021.0,1br,693.00,697.00,702.00,709.00,716.00,718.00,...,808.00,824.00,846.00,857.00,862.00,857.00,852.00,856.00,867.00,875.00
3,"Birmingham, AL",107000.0,212021.0,2br,816.00,821.00,827.00,834.00,843.00,846.00,...,952.00,970.00,996.00,1009.00,1015.00,1009.00,1004.00,1008.00,1021.00,1031.00
4,"Birmingham, AL",107000.0,212021.0,3br,1066.00,1072.00,1079.00,1089.00,1101.00,1104.00,...,1243.00,1267.00,1301.00,1317.00,1325.00,1317.00,1310.00,1317.00,1332.00,1346.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3817,"Wauwatosa, WI",5584675.0,47882.0,_Studio,NA,NA,NA,980.00,1004.00,992.00,...,960.00,999.00,1046.00,1058.00,1056.00,1020.00,997.00,1003.00,1015.00,1028.00
3818,"Wauwatosa, WI",5584675.0,47882.0,1br,NA,NA,NA,1140.00,1167.00,1153.00,...,1116.00,1162.00,1216.00,1230.00,1228.00,1186.00,1159.00,1166.00,1180.00,1195.00
3819,"Wauwatosa, WI",5584675.0,47882.0,2br,NA,NA,NA,1222.00,1250.00,1235.00,...,1196.00,1245.00,1303.00,1318.00,1316.00,1271.00,1242.00,1250.00,1265.00,1281.00
3820,"Wauwatosa, WI",5584675.0,47882.0,3br,NA,NA,NA,1495.00,1530.00,1512.00,...,1464.00,1524.00,1595.00,1613.00,1610.00,1556.00,1520.00,1530.00,1548.00,1568.00


In [ ]:
reframed_rent_df = pd.DataFrame(rent_df.set_index("RegionName").stack()).reset_index()
reframed_rent_df.columns = ["RegionName", "Date", "ZRent"]
reframed_rent_df.RegionName = reframed_rent_df.RegionName.str.replace(",", "")

# Apartments

In [3]:
import requests
from bs4 import BeautifulSoup
import csv
import time

import ApartmentsScraperHelper

bb = "xtqrw9_u9Isz8svW"
# pull the same code everytime but for San Marco
ApartmentsScraperHelper.main(bb = bb)

pd.read_csv(bb+".csv")

NameError: name 'csv' is not defined